### Will a Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaurant near where you are driving. Would you accept that coupon and take a short detour to the restaurant? Would you accept the coupon but use it on a subsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaurant? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \\$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\\$20 - \\$50). 

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece. 





### Data Description

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [233]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

### Problems

Use the prompts below to get started with your data analysis.  

1. Read in the `coupons.csv` file.

In [234]:
data = pd.read_csv('data/coupons.csv')

In [235]:
data.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


2. Investigate the dataset for missing or problematic data.

In [236]:
# Look for problematic data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

In [237]:
# Search for duplicated rows
data[data.duplicated()]

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
4192,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Male,26,Single,...,never,1~3,less1,less1,1,1,1,0,1,1
4236,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Male,26,Single,...,gt8,gt8,4~8,less1,1,1,1,0,1,1
4280,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,26,Single,...,never,4~8,1~3,less1,1,1,1,0,1,1
4324,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,46,Single,...,never,4~8,1~3,1~3,1,1,1,0,1,1
4409,Work,Alone,Sunny,80,7AM,Carry out & Take away,1d,Female,21,Single,...,never,less1,1~3,never,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,Home,Alone,Sunny,80,6PM,Bar,2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,0,0,1,0,1
8512,Home,Partner,Sunny,30,10PM,Carry out & Take away,2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,0,0,1,1
8513,Work,Alone,Rainy,55,7AM,Restaurant(<20),2h,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,1,0,1,0
8515,Work,Alone,Snowy,30,7AM,Restaurant(20-50),1d,Male,46,Married partner,...,1~3,1~3,less1,1~3,1,1,1,0,1,0


In [238]:
# Search for null values by column
data.isna().sum().sort_values(ascending=False)

car                     12576
CoffeeHouse               217
Restaurant20To50          189
CarryAway                 151
RestaurantLessThan20      130
Bar                       107
destination                 0
passanger                   0
direction_opp               0
direction_same              0
toCoupon_GEQ25min           0
toCoupon_GEQ15min           0
toCoupon_GEQ5min            0
income                      0
occupation                  0
education                   0
has_children                0
maritalStatus               0
age                         0
gender                      0
expiration                  0
coupon                      0
time                        0
temperature                 0
weather                     0
Y                           0
dtype: int64

In [239]:
# Explore columns with null values
data.value_counts('Bar', dropna=False)
data.value_counts('CoffeeHouse', dropna=False)

CoffeeHouse
less1    3385
1~3      3225
never    2962
4~8      1784
gt8      1111
NaN       217
dtype: int64

3. Decide what to do about your missing data -- drop, replace, other...

In [240]:
# Remove duplicates
data.drop_duplicates(keep='first', inplace=True)

In [241]:
# Cleaning up categorical columns by merging categories: less1, never, NaN
cleanup_dict = {
    'CoffeeHouse':          {'less1': '0', 'never': '0', pd.NA: '0'},
    'Restaurant20To50':     {'less1': '0', 'never': '0', pd.NA: '0'},
    'CarryAway':            {'less1': '0', 'never': '0', pd.NA: '0'},
    'RestaurantLessThan20': {'less1': '0', 'never': '0', pd.NA: '0'},
    'Bar':                  {'less1': '0', 'never': '0', pd.NA: '0'}
}
data.replace(cleanup_dict, inplace=True)

data['income'].replace(['-'], 'to', regex=True, inplace=True)

In [242]:
# Rename and transform column Y: Provide a more descriptive name and values
data = data.astype({"Y": str}, errors='raise')
data.rename(columns={'Y': 'Accepted'}, inplace=True)

cleanup_dict = {'Accepted': {'0': 'No', '1': 'Yes'}}
data.replace(cleanup_dict, inplace=True)

In [243]:
# Fix typo in column name by renaming column
data.rename(columns={'passanger': 'passenger'}, inplace=True)

In [244]:
# Convert column data type to more appropriate data types
print("Original dtypes:")
print(data.dtypes)

data = data.convert_dtypes()
print("New dtypes:")
print(data.dtypes)

Original dtypes:
destination             object
passenger               object
weather                 object
temperature              int64
time                    object
coupon                  object
expiration              object
gender                  object
age                     object
maritalStatus           object
has_children             int64
education               object
occupation              object
income                  object
car                     object
Bar                     object
CoffeeHouse             object
CarryAway               object
RestaurantLessThan20    object
Restaurant20To50        object
toCoupon_GEQ5min         int64
toCoupon_GEQ15min        int64
toCoupon_GEQ25min        int64
direction_same           int64
direction_opp            int64
Accepted                object
dtype: object
New dtypes:
destination             string
passenger               string
weather                 string
temperature              Int64
time                    str

4. What proportion of the total observations chose to accept the coupon?

In [245]:
# Get total number of rows
total_rows = len(data.index)

# Get value counts
df = data['Accepted'].value_counts()
ans1 = df['Yes']

# Get relative frequencies
df = data['Accepted'].value_counts(normalize=True) * 100
ans2 = df['Yes']

print('The proportion of drivers accepting the coupon is : {0} / {1} ( {2}% )'.format(ans1, total_rows, round(ans2)))

The proportion of drivers accepting the coupon is : 7157 / 12610 ( 57% )


In [246]:
# Plot Accepted vs Not Accepted
df = data.groupby('Accepted')['coupon'].count().reset_index(name='count').sort_values(by='count', ascending=False)
fig = px.bar(df,
             x='Accepted', y='count',
             color='Accepted',
             text_auto=True,
             title='Coupons offered to drivers: Accepted vs Not Accepted',
             color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

5. Use a bar plot to visualize the `coupon` column.

In [247]:
# Count coupons by coupon type
df = data.groupby('coupon')['coupon'].count().reset_index(name='count').sort_values(by='count', ascending=False)
fig = px.bar(df, x='coupon', y='count', color='coupon', text_auto=True, title="Coupons", color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

In [248]:
# Plot Histogram
fig = px.histogram(
    data,
    x='coupon', color='Accepted',
    text_auto=True,
    title='Coupon type: Accepted vs Not Accepted',
    color_discrete_sequence = px.colors.qualitative.Pastel
)

fig.show()

6. Use a histogram to visualize the temperature column.

In [249]:
# Plot histogram for temperature column
fig = px.histogram(data, x='temperature', nbins=20, text_auto=True)
fig.show()

**Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

1. Create a new `DataFrame` that contains just the bar coupons.


In [250]:
bar_coupons_df = data.query(" coupon == 'Bar' ")

2. What proportion of bar coupons were accepted?

In [251]:
# Get total bar coupons count
total_bar_coupons = len(bar_coupons_df.index)
accepted_coupons = bar_coupons_df.query(" Accepted=='Yes' ")['coupon'].size

# Get frequency
df = bar_coupons_df['Accepted'].value_counts(normalize=True) * 100
ans = df['Yes']

print('The proportion of bar coupons accepted : {0} / {1} ( {2} % )'.format(accepted_coupons, total_bar_coupons, round(ans)))

The proportion of bar coupons accepted : 824 / 2010 ( 41 % )


3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


In [252]:
# These variables will be used by points 3, 4, 5 and 6
accepted_bar_coupons_df =  bar_coupons_df.query(" Accepted=='Yes' ")
total_accepted = accepted_bar_coupons_df['coupon'].size

In [253]:
# Create two different groups for each criterion
rate_group_1 = round((accepted_bar_coupons_df.query(" Bar in ('0', '1~3') ")['coupon'].size / total_accepted) * 100)
rate_group_2 = round((accepted_bar_coupons_df.query(" Bar in ('4~8', 'gt8') ")['coupon'].size / total_accepted) * 100)

print('The acceptance rate for those who went to a bar 3 or fewer times a month is  : {0} %'.format(round(rate_group_1)))
print('The acceptance rate for those who went to a bar more than 3 times a month is : {0} %'.format(round(rate_group_2)))

The acceptance rate for those who went to a bar 3 or fewer times a month is  : 81 %
The acceptance rate for those who went to a bar more than 3 times a month is : 19 %


In [254]:
d = {'group': ['3 or fewer times', 'More than 3 times'], 'count': [rate_group_1, rate_group_2]}
df = pd.DataFrame(data=d)

fig = px.pie(df,
             values='count', names='group',
             title='Acceptance rate comparison for drivers who went to a bar',
             color_discrete_sequence = px.colors.qualitative.Pastel
             )
fig.update_traces(textposition='inside', textinfo='percent')
fig.show()

4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


In [255]:
query_string = " Bar in ('1~3', '4~8', 'gt8') and age in ('26', '31', '36', '41', '46', '50plus') "
rate_group = round((accepted_bar_coupons_df.query(query_string)['coupon'].size / total_accepted) * 100)

print('The acceptance rate is: {0} %'.format(round(rate_group)))
print('This group has a low acceptance rate compared to other groups.')

The acceptance rate is: 35 %
This group has a low acceptance rate compared to other groups.


5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry.

In [256]:
query_string = " Bar in ('1~3', '4~8', 'gt8') and passenger != 'Kid(s)' and occupation != 'Farming Fishing & Forestry' "
rate_group = round((accepted_bar_coupons_df.query(query_string)['coupon'].size / total_accepted) * 100)

print('The acceptance rate for this group is: {0} %'.format(round(rate_group)))

The acceptance rate for this group is: 48 %


6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K.

In [257]:
# Get rate for Group 1: Drivers who go to bars more than once a month, had passengers that were not a kid, and were not widowed.
query_1 = " Bar in ('1~3', '4~8', 'gt8') and passenger != 'Kid(s)' and maritalStatus != 'Widowed' "
rate_group_1 = round((len(accepted_bar_coupons_df.query(query_1).index) / total_accepted) * 100)

# Get rate for Group 2: Drivers who go to bars more than once a month and are under the age of 30.
query_2 = " Bar in ('1~3', '4~8', 'gt8') and age in ('below21', '21', '26') "
rate_group_2 = round((len(accepted_bar_coupons_df.query(query_2).index) / total_accepted) * 100)

# Get rate for Group 3: Drivers who go to cheap restaurants more than 4 times a month and income is less than 50K.
query_3 = " RestaurantLessThan20 in ('4~8', 'gt8') and income in ('Less than $12500', '$12500 - $24999', '$25000 - $37499', '$37500 - $49999') "
rate_group_3 = round((len(accepted_bar_coupons_df.query(query_3).index) / total_accepted) * 100)

print('The acceptance rate for Group 1 : {0} %'.format(round(rate_group_1)))
print('The acceptance rate for Group 2 : {0} %'.format(round(rate_group_2)))
print('The acceptance rate for Group 3 : {0} %'.format(round(rate_group_3)))

The acceptance rate for Group 1 : 48 %
The acceptance rate for Group 2 : 30 %
The acceptance rate for Group 3 : 4 %


In [258]:
d = {'group': ['Group 1', 'Group 2', 'Group 3'], 'count': [rate_group_1, rate_group_2, rate_group_3]}
df = pd.DataFrame(data=d)

group_desc = """
<br>Group 1: Go to bars more than once a month, had passengers that were not a kid, and were not widowed
<br>Group 2: Go to bars more than once a month and are under the age of 30
<br>Group 3: Go to cheap restaurants more than 4 times a month and income is less than 50K
"""

fig = px.bar(df,
             x='group', y='count', color='group',
             labels={'count': 'Acceptance rate'},
             text='count',
             color_discrete_sequence = px.colors.qualitative.Pastel)

fig.update_traces(texttemplate='%{text:.2s} %', textposition='outside')

fig.update_layout(
    title="Acceptance rate for different Groups",
    xaxis_title=group_desc,
    yaxis_title="Acceptance rate",
    legend_title="Groups",
    font=dict(
        size=12,
    )
)

fig.show()

7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

### Findings
- Drivers who go to a bar 3 or fewer times have a higher probability of accepting a bar coupon than those who go more than 3 times.
- Not having kids as passengers while driving increases the probability of accepting a bar coupon.
- Drivers who go to cheap restaurants have a low acceptance rate of bar coupons.

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.

In [259]:
# This investigation will be focused in the coffee coupons group
coffee_coupons_df = data.query(" coupon=='Coffee House' ")
accepted_coffee_coupons_df = coffee_coupons_df.query(" Accepted=='Yes' ")

1. Compare the acceptance rate between male and female drivers for coffee coupons.

In [260]:
# Plot acceptance rate: Male vs Female
df = accepted_coffee_coupons_df.groupby('gender')['coupon'].count().reset_index(name='count')

fig = px.pie(df,
             values='count', names='gender',
             title="Coffee coupons acceptance rate: Male vs Female",
             color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

2. Identify acceptance rate by age.

In [261]:
# Acceptance rate by age
df = accepted_coffee_coupons_df.groupby('age')['coupon'].count().reset_index(name='count')
fig = px.pie(df,
             values='count', names='age',
             title="Coffee coupons: acceptance rate by age",
             color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

3. Identify the top 3 occupation groups with the highest coffee coupons acceptance.

In [262]:
# Plot the top 3 occupation groups accepting coffee coupons
df = accepted_coffee_coupons_df.groupby('occupation')['coupon'].count().reset_index(name='count').sort_values(by='count', ascending=False).head(3)
fig = px.bar(df,
             y='count', x='occupation', color='occupation',
             title="Coffee coupons acceptance: Top 3 occupations",
             color_discrete_sequence = px.colors.qualitative.Pastel)
fig.show()

4. Identity the coffee coupon acceptance rate by income bracket.

In [263]:
df = accepted_coffee_coupons_df.groupby('income')['coupon'].count().reset_index(name='count').sort_values(by='count', ascending=False) #.head(3)
fig = px.pie(df,
             values='count', names='income',
             title="Coffee coupons acceptance rate by income bracket",
             color_discrete_sequence = px.colors.qualitative.Pastel)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

5. Compare acceptance rate by gender and weather.

In [264]:
df = coffee_coupons_df.groupby(['gender', 'Accepted', 'weather'])['coupon'].count().reset_index(name='count')

In [265]:
fig = px.bar(df,
             x='weather', y='count',
             color='Accepted',
             barmode='group',
             facet_col='gender',
             title='Coffee coupon acceptance by gender and weather.',
             category_orders={'weather': ['Sunny', 'Rainy', 'Snowy'],
                              'Accepted': ['Yes', 'No'],
                              'sex': ['Male', 'Female']})
fig.show()

### Findings
- 21 years old group has the highest acceptance rate than any other group.
- Drivers who reported low income are more likely to accepts coffee coupons.
- Students and those who have computer and mathematical occupations are the top groups with highest rate acceptance than other groups.